[← Back to main notebook](https://colab.research.google.com/github/lrhgit/viscoelastic-fluids-demo/blob/main/intro_viscoelasticity.ipynb)


# Comparison — Maxwell vs Kelvin–Voigt

This notebook overlays the **creep** and **stress relaxation** responses of the two classical linear viscoelastic models:
- **Maxwell** (series): captures stress relaxation and unbounded creep.
- **Kelvin–Voigt** (parallel): captures bounded creep and no stress relaxation under a step strain.

All notation is consistent: $E$ (elastic modulus), $ \eta $ (viscosity), and $ \tau = \eta/E $ (characteristic time).


In [10]:
#@title ⚙️ Imports and setup (click to expand)

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
def stress_relaxation_maxwell(t, E, eta, gamma0):
    tau = eta / E
    return E * gamma0 * np.exp(-t / tau)
def creep_maxwell(t, E, eta, sigma0):
    return sigma0 / E + (sigma0 / eta) * t
def creep_kv(t, E, eta, sigma0):
    tau = eta / E
    return (sigma0 / E) * (1 - np.exp(-t / tau))
def stress_relaxation_kv(t, E, eta, gamma0):
    return np.ones_like(t) * E * gamma0
# Sliders (two columns)
E_slider = widgets.FloatLogSlider(value=1e5, base=10, min=3, max=7, step=0.1, description='E [Pa]')
eta_slider = widgets.FloatLogSlider(value=1e6, base=10, min=4, max=8, step=0.1, description='η [Pa·s]')
sigma0_slider = widgets.FloatSlider(value=100, min=10, max=1000, step=10, description='σ₀ [Pa]')
gamma0_slider = widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='γ₀')
T_slider = widgets.FloatLogSlider(value=10, base=10, min=-1, max=3, step=0.1, description='T_process [s]')
tmax_slider = widgets.FloatLogSlider(value=10, base=10, min=-2, max=4, step=0.1, description='t_max [s]')
ui_left = VBox([E_slider, eta_slider, sigma0_slider])
ui_right = VBox([gamma0_slider, T_slider, tmax_slider])
ui = HBox([ui_left, ui_right])
def plot_comparison(E, eta, sigma0, gamma0, T_process, tmax):
    tau = eta / E
    De = tau / T_process
    t = np.linspace(0, tmax, 400)
    # Creep
    gamma_M = creep_maxwell(t, E, eta, sigma0)
    gamma_KV = creep_kv(t, E, eta, sigma0)
    # Stress relaxation
    sigma_M = stress_relaxation_maxwell(t, E, eta, gamma0)
    sigma_KV = stress_relaxation_kv(t, E, eta, gamma0)
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    # Left: creep (overlay)
    ax[0].plot(t, gamma_M, lw=2, label='Maxwell')
    ax[0].plot(t, gamma_KV, lw=2, label='Kelvin–Voigt')
    ax[0].set_title('Creep — Overlay')
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Strain γ [-]')
    ax[0].grid(True)
    ax[0].legend()
    # Right: stress relaxation (overlay)
    ax[1].plot(t, sigma_M, lw=2, label='Maxwell')
    ax[1].plot(t, sigma_KV, lw=2, label='Kelvin–Voigt')
    ax[1].set_title('Stress Relaxation — Overlay')
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Stress σ [Pa]')
    ax[1].grid(True)
    ax[1].legend()
    plt.tight_layout()
    plt.show()
    print(f"τ = {tau:.3g} s  |  De = {De:.3g}")
out = widgets.interactive_output(
    plot_comparison,
    {'E': E_slider, 'eta': eta_slider, 'sigma0': sigma0_slider, 'gamma0': gamma0_slider, 'T_process': T_slider, 'tmax': tmax_slider}
)
display(ui, out)


Output()


---

## Discussion
- **Creep:** Maxwell shows linear increase of strain; Kelvin–Voigt approaches a finite limit $\sigma_0/E$.  
- **Stress relaxation:** Maxwell decays exponentially; Kelvin–Voigt stays constant at $E\gamma_0$.  
- The common timescale $ \tau = \eta/E $ governs how fast changes occur; compare it to the process time $T_\text{process}$ via $De=\tau/T_\text{process}$.



---

## 👩‍🏫 Teacher Information (Comparison)
Use this notebook to **overlay** both models and guide discussion:
1. Start with the same $E$ and $\eta$; vary $\sigma_0$ and $\gamma_0$ to see differences in shape.
2. Emphasize that both models are *linear* but encode different topology (series vs parallel).
3. Connect $ \tau $ and $ De $ to real processes (e.g., dwell time in a rheometer, loading rate in a test).

Tip: ask students to identify parameters where the two models look deceptively similar (short $t_{max}$, small $De$) and where they clearly diverge.
